# Метод простой итерации. Метод Зейделя

In [1]:
import numpy as np
from math import sqrt
import sys
from scipy.linalg import hilbert
from tabulate import tabulate
from tqdm import tqdm

# В качестве тестовых данных используются

In [2]:
# 1. Матрица Гильберта второго порядка
matrixHilbert2 = np.array(hilbert(2))
print('\n'.join(' '.join(str(cell) for cell in row) for row in matrixHilbert2))
print()

# 2. Матрица с диагональным преобладанием
matrixD = np.array([[50, 10, 20], [70, 82, -14], [5, -11, 20]])  
print('\n'.join(' '.join(str(cell) for cell in row) for row in matrixD))
print()

# 3. Матрица из методички Пакулиной
matrix1 = np.array([[-400.60, 199.80], [1198.80, -600.40]])  
print('\n'.join(' '.join(str(cell) for cell in row) for row in matrix1))
print()
                    
# 4. Матрица из методички Пакулиной
matrix2 = np.array([[-402.90, 200.70], [1204.20, -603.60]])  
print('\n'.join(' '.join(str(cell) for cell in row) for row in matrix2))
print()

1.0 0.5
0.5 0.3333333333333333

50 10 20
70 82 -14
5 -11 20

-400.6 199.8
1198.8 -600.4

-402.9 200.7
1204.2 -603.6



### Нахождение b

In [3]:
def findB(A, x):
    b = np.dot(A, x)
    return b

### Находение alpha и beta

In [4]:
def findCoef(A, b):  
    alpha = np.array(np.zeros((A.shape[0], A.shape[0])))
    beta = np.array(np.zeros(b.shape[0]))
    for i in range(A.shape[0]):
        for j in range(A.shape[0]):
            if i != j:
                alpha[i][j] = - A[i][j] / A[i][i]
                beta[i] = b[i] / A[i][i]
            else:
                alpha[i][i] = 0
    return alpha, beta

### Метод простой итерации

In [5]:
def iterationMethod(alpha, beta, x, eps):
    iter = 1
    err = eps + 1
    while err > eps:
        err = np.linalg.norm(np.dot(alpha, x) + beta - x)
        x = np.dot(alpha, x) + beta
        iter += 1
    x = np.dot(alpha, x) + beta
    return x, iter

### Метод Зейделя

In [6]:
def seidelMethod(A, b, eps):
    iter = 0
    x = np.array(np.zeros((b.shape[0])))
    err = eps + 1
    while err > eps:
        xNew = x.copy()
        for i in range(A.shape[0]):
            x1 = sum(A[i][j] * xNew[j] for j in range(i))
            x2 = sum(A[i][j] * x[j] for j in range(i + 1, A.shape[0]))
            xNew[i] = (b[i] - x1 - x2)/A[i][i]
        err = np.linalg.norm(xNew - x)
        iter += 1
        x = xNew
    return x, iter

### Вывод данных

In [7]:
print("Матрица:")
A = matrix2
print(*A, sep='\n')
x = np.random.uniform(0, 100, size=A.shape[0])

b = findB(A, x)
alpha, beta = findCoef(A, b)
for eps in (1e-4, 1e-7, 1e-10, 1e-13):
    print("Погрешность:", eps)
    print("Методе простой итерации:")
    print("   Количество итераций:", iterationMethod(alpha, beta, beta, eps)[1])
    print("   ||x - x_a||:", np.linalg.norm(x - iterationMethod(alpha, beta, beta, eps)[0]))
    print("Метод Зейделя:")
    print("   Количество итераций:", seidelMethod(A, b, eps)[1])
    print("   ||x - x_a||:", np.linalg.norm(x - seidelMethod(A, b, eps)[0]))
    
    print()
    print()

Матрица:
[-402.9  200.7]
[1204.2 -603.6]
Погрешность: 0.0001
Методе простой итерации:
   Количество итераций: 4039
   ||x - x_a||: 0.00031960064961886935
Метод Зейделя:
   Количество итераций: 1379
   ||x - x_a||: 0.016006343824159015


Погрешность: 1e-07
Методе простой итерации:
   Количество итераций: 6261
   ||x - x_a||: 3.194094128556873e-07
Метод Зейделя:
   Количество итераций: 2490
   ||x - x_a||: 1.5996801329642467e-05


Погрешность: 1e-10
Методе простой итерации:
   Количество итераций: 8483
   ||x - x_a||: 3.185110595573772e-10
Метод Зейделя:
   Количество итераций: 3601
   ||x - x_a||: 1.5986781127385062e-08


Погрешность: 1e-13
Методе простой итерации:
   Количество итераций: 9901
   ||x - x_a||: 3.7783038986514795e-12
Метод Зейделя:
   Количество итераций: 4694
   ||x - x_a||: 1.7327701673394473e-11




## Метод релаксации

In [8]:
def relaxMethod(A, b, x):
    n = A.shape[0]
    x = x.copy()
    mask = np.full(n, True)
    for j in range(n):
        deltas = A @ x - b
        k = np.nanargmax(np.abs(np.where(mask, deltas, np.nan)))
        if A[k, j] != 0:
            x[j] = (b[k] - A[k] @ x + A[k, j]*x[j]) / A[k, j] 
        mask[k] = False
    return x

In [9]:
def relaxSolver(A, b, x, eps):
    iter = 1
    err = eps + 1
    while err > eps:
        x_new = relaxMethod(A, b, x)
        err = np.linalg.norm(x - x_new)
        iter += 1
        x = x_new
    return x, iter

In [10]:
def test():
    n = 201
    A = np.random.rand(n, n)
    for i in range(n):
        A[i, i] = sum([abs(A[i, j]) for j in range(n)])
    return A

In [11]:
def varirovanie(value, eps):
    return np.random.choice([-eps, eps], size=value.shape) + value

In [12]:
def varirovanie_2():
    return np.random.uniform(0, 201, size=A.shape[0])

In [13]:
test().shape

(201, 201)

In [18]:
A = test()
x = np.random.uniform(0, 201, size=A.shape[0])
b = findB(A, x)
for i in range(1000):
    A_2 = varirovanie(A, 1e-4)
    for j in range(1000):
        try:
            start_x = varirovanie_2()
            x_2 = relaxSolver(A_2, b, start_x, 1e-5)
            print('Решение найдено!')
            print('Норма разности', np.linalg.norm(x_2 - x))
            break
        except KeyboardInterrupt:
            raise KeyboardInterrupt()
        except:
            pass

<ipython-input-8-ebb96acb7a02>:9: RuntimeWarning: overflow encountered in matmul
  x[j] = (b[k] - A[k] @ x + A[k, j]*x[j]) / A[k, j]
<ipython-input-8-ebb96acb7a02>:6: RuntimeWarning: invalid value encountered in matmul
  deltas = A @ x - b
<ipython-input-8-ebb96acb7a02>:9: RuntimeWarning: overflow encountered in double_scalars
  x[j] = (b[k] - A[k] @ x + A[k, j]*x[j]) / A[k, j]


Решение найдено!
Норма разности 7.53982713e-07
